# Salary estimator from listings

## Introduction

In this notebook we will use the [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings) kaggle dataset to estimate job pay based on a job title and the state it is in. As we analyze the data, we will use the [Plotly](https://plotly.com/python/) library for visualizations. We will train the extreme gradient boosted decision trees machine learning algorithm to estimate the job pay. We will use the XGBRegressor model from the [DMLC XGBoost](https://xgboost.readthedocs.io/) library. That XGBoost model only allows numeric inputs. We will use the [Sci-kit Learn](https://scikit-learn.org/) library to pipe our data through transformers that translate the job titles and states into numbers and interact with XGBoost. We will use the Sci-kit Learn OneHotEncoder to transform the state. Since the job titles have such a high cardinality, we need to represent them differently. So, we will use the [Gensim](https://radimrehurek.com/gensim/) library Word2Vec model to create word embedding vectors representing the job titles for XGBoost. After training XGBRegressor, we will use Ipython to make a small user interface to estimate the job pay based on the job title and state.

Click `Runtime` > `Run all` or press `ctrl` + `F9`. Then click `Run anyway.` On Google Colab, this usually takes around five minutes.

You can skip to where it makes estimations or go through each section to understand what is happening in the application. If you want to skip to the estimation section, scroll to the bottom block to read the instructions and use the model.

## Setup

To begin, we will install all the necessary packages for the application to run. We can do that by running the block of code below that uses IPython commands to install the packages.

In [ ]:
%pip install -q "scipy==1.12"  gensim xgboost scikit-learn pyarrow plotly Jinja2 nbformat ipywidgets pandas gdown

Next, we retrieve the data and the rest of the code.

In [ ]:
import os
dir_path = os.path.abspath('.')    
import shutil


def rm_files(root, dir_path):
    for x in os.listdir(dir_path):
        xname = '/'+x
        if os.path.isfile(dir_path+xname):
            os.remove(dir_path+xname)
        elif x != '.config' and os.path.isdir(dir_path+xname):
            rm_files(root, dir_path+xname)
    if root != dir_path:
        os.rmdir(dir_path)


def extract_files(root, dir_path):
    for x in os.listdir(dir_path):
        xname = '/'+x
        if root != dir_path and x[0] != '.':
            shutil.move(dir_path+xname, root+xname)
        elif os.path.isdir(dir_path+xname):
            rm_files(root, dir_path+xname)
        else:
          os.remove(dir_path+xname)


if not os.path.exists(dir_path+'/settings.py'):
    !git clone --depth 1 https://github.com/Trail3lazer/pay-estimator.git
    import gdown
    url = 'https://drive.google.com/drive/folders/1ZTZDh1TPjUOBy7sAruyIt567GFDHQ2C-?usp=sharing'
    output = dir_path+'/pay-estimator'
    gdown.download_folder(url=url, output=output, quiet=False)
    extract_files(dir_path, output)
    os.rmdir(output)
    
    
import settings
if not os.path.exists(settings.APP_ARCHIVE_PATH):
    os.makedirs(settings.APP_ARCHIVE_PATH)

## Parsing and cleaning the data

After we set up our supporting code and data, we can start parsing the data! First, we will instantiate the DataManager class. It will handle reading the CSV files, joining the data, dropping unnecessary columns, renaming confusing columns, parsing which state the jobs are in, getting the average pay for each job, transforming different pay periods to yearly pay for consistency, dropping duplicates, and saving the cleaned, transformed job postings.

Run the code block below to create the DataManager class and clean the data.

In [ ]:
from data import DataManager

dm = DataManager()
df = dm.get_postings()

Now that we have cleaned the data, we will review a few postings. We will shorten some text columns to fit most of the rows on the screen vertically. You can use the horizontal scroll bar to see more columns.

Run the code block below to see a sample set of the cleaned data.

In [ ]:
from IPython.display import HTML, display

df = dm.get_postings()
def shorten_long_cols(row):
    for name in ['job_desc','company_desc','skills_desc']:
        if isinstance(row[name], str):
            row[name] = row[name][:150] + '...' 
    return row

display(HTML(df.sample(2).apply(shorten_long_cols, axis=1).to_html()))

There are a few important items we should pay attention to. 

1. The location column, which is the location of the job, is not normalized to a state abbreviation. Usually, it's in the format "City, State," but that is not always the case. The state column originally represented the location of the company. Many jobs are in states other than where the company HQ resides. So we use some regex and code in the DataManager to extract the state and translate it to a state abbreviation, then save it to the state column. We use the company state column if we can't extract a state from the location column. 

2. Job titles are generally unique. Their uniqueness could cause some issues when we try to use regression to estimate salaries for job titles. If the XGBoost model receives data with high cardinality, it will be much less accurate. So, we need to create word embeddings to represent the job titles and reduce their complexity. We will use the Word2Vec model to generate vectors from words and use the vectors to represent the job titles with XGBoost.

3. To ensure the accuracy of the vectors, we will use a substantial amount of text data, including The job_title, job_desc, job_skills, skills_desc, and company_desc, to train the word2vec model. We need a lot of data to make the vectors, so we can use the plethora of text from these columns to train it.

4. The pay_period and salary columns include data for different pay periods. If we train the model with inconsistent pay information, it will not be able to estimate pay accurately. We needed to normalize those columns. So, the DataManager class converts all pay periods to yearly pay. For most pay periods, it multiplies the pay to represent a year's pay. e.g., monthly pay times twelve equals annual pay. We must account for time off, full-time/part-time hours, and holidays for hourly pay. The DataManager uses statistics from the Bureau of Labor Statistics to calculate the average full-time and part-time working hours per week and working weeks per year. Then, it multiplies those with the pay to create the job's yearly pay.

5. It also takes the average between the max, med, and min salary columns if they exist. The salary columns having _from_salaries at the end of their names come from the salaries CSV file. So, if the primary salary columns are empty, the DataManager uses the backup columns to calculate the pay. After normalizing the pay columns, the DataManager saves the results to the "pay" column.

## Data Analysis

Run the code block below to create descriptive statistics for the cleaned data.

In [ ]:
pay_cols = ['max_salary','med_salary','min_salary','pay']
pay_period_df = dm.get_postings()[pay_cols]
desc = pay_period_df.describe()

message = '''
<div style="font-size:17px">
    <p>
        The pay statistics have a few interesting features. 
        First, the "pay" column is a calculated column the DataManager created earlier. 
        It represents the average for all pay columns in each row.
    </p>
'''
for col in pay_cols:
    message += r'''<p>
    There are {count:,.0f} {col} values. The average {col} is &dollar;{mean:,.0f}, and the standard deviation is {std:,.0f}. 
    The {col} column has a minimum of &dollar;{min:,.0f}, a median of &dollar;{50%:,.0f}, and a maximum of &dollar;{max:,.0f}. 
    25&percnt; of the {col} values are less than &dollar;{25%:,.0f}, and 25&percnt; are greater than &dollar;{75%:,.0f}. 
    So, 50&percnt; of the values are between &dollar;{25%:,.0f} and &dollar;{75%:,.0f}.
    </p>'''.format(col=col, **desc[col])

message += '''<p>
It's important to note that the column values are not all filled out. 
We have more max and min salary entries than med salary entries. 
This difference suggests potential data gaps that need further investigation. 
Additionally, many jobs are missing pay information, as the number of rows with pay values is significantly lower than the total number of jobs.
</p></div>'''
display(HTML(desc.style.format(precision=0,thousands=",").to_html()))
display(HTML(message))

Next, we will create some box pots to visualize the distribution of the pay columns. We will use a graphing library called Plotly to display our data.

Run the code block below to create the boxplots.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(layout=dict(height=800, title='Job Posting Pay Column Box Plots'))
fig.add_traces([go.Box(y=pay_period_df[col], boxpoints=False, name=col) for col in pay_cols])
fig.update_yaxes(tickprefix = '$', tickformat = ',.0f', type="log",automargin=True)
fig.show()

The box plots above represent the descriptive statistics we generated earlier.

Next, we will create a bar graph of average yearly pay by state. Run the code block below to generate the bar graph.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = dm.get_postings()[['state','pay']].copy().dropna(how='any')

groups = df.groupby('state')
group_count = groups.count()
df = groups.mean()
df['count'] = group_count
df = df.dropna(axis=1).sort_values(by='pay')

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x = df.index.values, 
        y=df['pay'],
        name="Average Yearly Pay",
    ), 
    secondary_y=False)

fig.add_trace(
    go.Scatter(
        x = df.index.values,
        y = df['count'],
        name="Sample Size"
    ),
    secondary_y=True
)

fig.update_layout(
    title=dict(text="Average Job Posting Pay By State"),
    xaxis=dict(title_text="State",tickangle=90),
    yaxis=dict(title_text="Average Yearly Pay")
)

fig.update_yaxes(title_text="Job Listings (log)", secondary_y=True, type="log")

fig.show()

The bars represent the average yearly pay in each state. The red line represents the number of job listings for that state. Some states have very few job listings. Our pay estimates in those states will be less accurate. Specifically, Wyoming has the least job listings at 96.

Additionally, the states with the most listings will skew the overall dataset statistics. California has the most listings at 6615. The state with the minimum average pay is Mississippi, with an average income of &dollar;56,768.58. Washington, DC, has the highest average income at &dollar;111,351.10.

## Word embedding model

Now, we should see what the average pay is for different jobs. Since the job titles have a high cardinality, we can not reasonably display them on an axis in a chart. So before we try to get the averages, we need to sort the job postings into categories. To do that, we will create word embeddings with Word2Vec. The class called Job2Vec prepares training data and trains and manages the Word2Vec model.

Run the code block below to create the Job2Vec class.

In [ ]:
from wordmod import Job2Vec

job2vec = Job2Vec()

It takes a while to train the Word2Vec model, so I saved it in the assets folder. We will reuse the trained model if possible.

If we have to train a new model, we will tokenize all the text data and then pass it to the Job2Vec class to train it. The Job2Vec class will save the tokenized strings to the "archive/app" folder because tokenizing the text takes a long time. 

Run the code block below to get or train the Word2Vec model.

In [ ]:
import numpy as np

print('Loading j2v word vectors.')
j2v = job2vec.try_get_model()

if j2v is None:
    dataset = job2vec.try_load_dataset()

    if dataset is None:
        df = dm.get_postings().copy()

        print("Combining the the bls.gov job list, LinkedIn job title, description and skills, columns, and other tables to create a single array. Word2Vec does not need them separated.")
        bls = dm.get_bls_jobs().to_numpy()
        others = np.concatenate(dm.load_additional_tables())
        data = [bls, others, df['job_title'].unique(), df['job_desc'].unique(), df['skills_desc'].unique(), df['company_desc'].unique()]
        ser = np.concatenate(data)

        dataset = job2vec.preprocess_data(ser)
    
    j2v = job2vec.try_get_model(dataset)


Now that the word embedding model is loaded and trained, we can use the word vectors to create categories and categorize the jobs. I made a categorizer class that will take a list of categories and create vectors for each category. Then, we can use the original model to vectorize job titles and use the category vectors to determine the most similar category to the job title. First, we will instantiate the Categorizer class.

Run the code block below to create the Categorizer class.

In [ ]:
from catword import Categorizer
import pandas as pd

categorizer = Categorizer(j2v.wv, job2vec.tokenize)

The data manager can use the categorize function from the categorizer class to categorize each job title and save the result to a category column. So, we will have the DataManager categorize our job titles.

In [ ]:
df = dm.get_or_create_categorized_postings(categorizer.categorize)

## More Data Analysis

Now that the job titles are classified, we will generate another bar chart to show the average pay for each job category. Run the code block below to create the job category pay bar chart.

In [ ]:
groups = df[['pay', 'category']].dropna(how='all').groupby('category')
fig_df = groups.mean(numeric_only=True).sort_values(by='pay')
fig_df['count'] = groups.count()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig = fig.add_trace(go.Bar(x=fig_df.index.values, y=fig_df['pay'], name='Average Pay'))

fig.update_layout(
    title=dict(text="Average Job Posting Pay By Category"),
    yaxis=dict(title_text="Average Yearly Pay")
)

fig.add_trace(
    go.Scatter(
        x = fig_df.index.values,
        y = fig_df['count'],
        name="Sample Size"
    ),
    secondary_y=True
)

fig.update_yaxes(title_text="Job Listings (log)", secondary_y=True, type="log")

fig.show()

This bar chart is laid out similarly to the Average Job Posting Pay by State chart. The red line shows the number of jobs in each category. The category with the most jobs is business, with 6482 job listings. The category with the fewest listings is environment, with 48 listings. The periwinkle bars show the average pay for each category. The category with the highest average salary is information technology. The categories with the lowest average pay are retail and grocery.

Now that we have all the jobs categorized, we can create a heat map that shows which states have the most and least average pay for each category. Some states do not have jobs in different categories, so those squares are blank. The more yellow a square is, the higher its average pay.

Run the code block below to create the heatmap.

In [ ]:
import plotly.express as px

df = dm.get_or_create_categorized_postings(categorizer.categorize).copy()
fig_df = df.copy()
fig_df = fig_df[['category','state','pay']]
fig_df = fig_df.groupby(by=['category','state'], group_keys=False).mean().round(2)
fig_df.name = 'pay'
heat_df = fig_df.unstack(level=1)
heat_df.columns = heat_df.columns.droplevel(0)
fig = px.imshow(heat_df, aspect="auto", height=700)
fig.show()

This heat map represents the average pay for each job category in each state. The more yellow a cell is, the higher the average salary is. Conversely, if a cell is more purple, it represents lower pay. Interestingly, some blocks on the map are empty. The state has no listings in that job category if the blocks are empty. It's interesting to see some columns and rows are darker than others. It's another representation of what we saw earlier, where some states and job categories pay better than others.

Next, we will create a 3D scatter plot of the same data points. It will be fun to visualize these same ideas in a different way. 

Run the code block below to create the 3D scatter plot.

In [ ]:
df = fig_df.reset_index()
df['color'] = 0
df.loc[df['pay'] >= 200_000,'color'] = 200_000
df.loc[(200_000 > df['pay']) & (df['pay'] >= 150_000),'color'] = 150_000
df.loc[(150_000 > df['pay']) & (df['pay'] >= 100_000),'color'] = 100_000
df.loc[(100_000 > df['pay']) & (df['pay'] >= 50_000),'color'] = 50_000
df.loc[50_000 > df['pay'],'color'] = 0

fig = px.scatter_3d(df, x='state', y='category', z='pay', color='color')
fig.update_layout(
    height=800,
    xaxis=dict(nticks=51),
    yaxis=dict(nticks=51),
    margin={"t":0,"b":0}
)
fig.update_scenes(aspectmode='cube')
fig.show()

Awesome! This graph is more difficult to read than the heat map, but it's much more fun. You can click and drag your mouse around the graph window, which will rotate. Additionally, you can scroll in the graph window to zoom in and out. If you hover over different markers, you'll see the data they represent. The data points are the same as the heat map, but it's interesting to see the distribution of job pay by the colors and groupings of the points and a 3D space.

## XGBoost training

Now, we can get to work on the estimation model. We will use the XGBoost XGBRegressor class to predict the pay for different job titles in various states. First, we must filter out all the jobs where the state, job title, or pay are invalid. After we filter the columns, we will create two distinct data frames. One will be our training data, and the other will be our target data. The training data will include the state and job title. The target data will consist of the pay. They are labeled X and Y. Then we can pass those to the Sci-kit Learn train test split method, which will split our data into a training and testing set. We will use a test size of 0.1.

Run the code block below to create the training and testing datasets.

In [ ]:
from sklearn.model_selection import train_test_split
df = dm.get_postings().copy()
x_cols=['state',
        'job_title']
y_col = 'pay'

mask = df[['job_title', 'state', y_col]].notna().all(axis=1) & df[y_col].gt(0)
df = df[x_cols+[y_col]].loc[mask].copy().reset_index()

x, y = df[x_cols], df[y_col]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.1)

After splitting our training data set, we can prepare a pipeline to transform the data for xgboost. XGBoost only accepts numerical data, so we must turn our string values into numbers. We could use a map of numbers to the values. For example, we could map AK to 1, AL to 2, AR to 3, etc. The issue with doing that is that XGBoost may think that the states with the higher numbers are more valuable in some way. That could cause issues where a state like Wyoming, which comes in the end, could have much higher predictions than it should. A common way to solve this is to use One Hot Encoding. One Hot Encoding splits each state into its column and uses one or zero to note if that is the state for the row. For example, let's say we have a row in which the state is CA. Then the row may look like this:

| job_title | ... | AR | AZ | CA | CO | ... |
| --------- | --- | -- | -- | -- | -- | --- |
| "Some job"| ... | 0  | 0  | 1  | 0  | ... |

Since only 52 states (including DC) exist, One Hot Encoding should work great.

With the job titles, we will run into the same problem we had earlier because the job titles have a high cardinality. In other words, there are many different job titles, but there are far too many to split them up meaningfully with One Hot Encoding. We could categorize the jobs like we did earlier, but then the model can only estimate based on job categories. We want it to calculate pay for many jobs, not just job categories. Luckily, the Word2Vec model we created uses numerical vectors to represent words. We can represent job titles by retrieving the vectors for each word in the job title and calculating the average from them. The Job2Vec class will calculate the mean vector for the job titles. We just need to pass the job title to its vectorize method. So, we will create a custom functional transformer that vectorizes the job titles in our dataset.

Now that we know what to do with each column, we can create a pipeline that handles each column individually. After we make the column transformer, we must fit the transformer with our training data.

Run the code block below to create the data preprocessor.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer


vector_length = j2v.wv.vector_size

def title_to_vec(titles: pd.DataFrame):
    vector_cols = [f'title{n}' for n in range(vector_length)]
    rows = [job2vec.vectorize(x) for x in titles['job_title'].values]
    return pd.DataFrame(rows, columns=vector_cols)

preprocessor = ColumnTransformer(transformers=[
    ("state", OneHotEncoder(handle_unknown="ignore"), ['state']),
    ("title", FunctionTransformer(title_to_vec), ['job_title'])
])

preprocessor = preprocessor.fit(x,y)

After we fit the preprocessor, we can load or train the XGBoost model. We will use another Sci-kit Learn pipeline to easily manage passing data through our preprocessor and the XGBoost regressor model. We will instantiate the XGBRegressor class and use the gbtree booster, hist tree method, regression squared error objective, and mean average error metric in the hyperparameters. There are additional methods I included. The eta max depth N estimators I have set produced the most accurate results from my testing. Reducing the max depth made the model too simplistic to handle the word vectors and the one hot encoded states. Additionally, adding so many trees via the n_estimators parameter helped fine-tune the model.

After we create the pipeline and instantiate the regressor class, we can either load my existing model or train a new one. Training such complex decision trees takes a lot of time, processing, and computing power, even with a GPU. Using the model included instead of retraining a new one.

Run the code block below to create and load or train the XGBRegressor model and pipeline.

In [ ]:
import xgboost as xgb
import os, settings

xgb_reg: xgb.XGBRegressor = xgb.XGBRegressor(
    device='cuda',
    booster='gbtree',
    tree_method= 'hist',
    objective='reg:squarederror',
    eval_metric='mae',
    eta=0.1, 
    max_depth=20,
    early_stopping_rounds=15,
    verbosity=0,
    n_estimators=500
)

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocessor),
    ('reg', xgb_reg)
])

if os.path.isfile(settings.XGB_MODEL):
    xgb_reg.load_model(settings.XGB_MODEL)
    
else:
    x_test_preprocessed = preprocessor.transform(x_test)
    
    xgb_pipe = xgb_pipe.fit(
        x_train, 
        y_train, 
        reg__eval_set=[(x_test_preprocessed, y_test)])
    
    xgb_reg.save_model(settings.XGB_MODEL)

## Validation

Now that our model is trained and loaded, we must test its accuracy. We can do that using our test set. We'll create a small sample of ten different jobs, their actual pay, and what the model estimates the pay to be, just for our reference to see visually how the model performs. Then, we can have the model predict the entire test set and calculate the mean average percentage error (MAPE) to see its estimation accuracy. The mean average percentage error works well with many regression models and helps normalize error rates. So, the mean average percentage error will tell us approximately how inaccurate the model's salary estimation is. It will cover validation for the Word2Vec and XGBoost models because both play a role in creating the estimation through our pipeline.

Run the code block below to test the model and calculate the accuracy metric.

In [ ]:
test = pd.DataFrame(x_test, columns=['state','job_title'])
test['pay'] = y_test
sm_test = test.sample(5)
res = xgb_pipe.predict(sm_test[x_cols])
sm_test['predicted']=res
display(HTML(sm_test.style.format(precision=2,thousands=",").to_html()))

mape = ((test['pay']-xgb_pipe.predict(test[x_cols]))/test['pay']).abs().mean()
display(HTML(f'''
             <p style="font-size:18px;color:orange;font-weight:bold">
                    MAPE = {mape*100:.10f}&percnt;
             </p>
             '''))

## Try It Out

The code below creates a simple UI that allows us to enter job information and get yearly and hourly pay estimates. We can use the UI to type in a job title or description and select the state where we want it to estimate the pay. It also displays words similar to the job title we are entering to help us determine if the model correctly understands what we are typing. That way, if we are trying to estimate the pay for a mental health counselor, but the words that pop up include words similar to legal counsel, it is a good indicator that the model may estimate the incorrect pay. In a case like that, we could slightly change the words we enter for the job title to be more unique to that job. So, instead of a mental health counselor, we could enter a mental health therapist. Sometimes, including additional details about the job helps the model be more accurate.
Additionally, since we trained the model using a LinkedIn job posting data set, it won't know about jobs that rarely appear on LinkedIn or are not posted on LinkedIn. The model will incorrectly estimate the pay for those jobs because it is not trained by the data to know the correct pay for those jobs. Some examples of jobs it might not understand could include actor, entrepreneur, or farmer.

Run the code block below and try typing different job titles into the UI to determine their pay. Some fun ideas could include 'dietitian,' 'mechanic,' 'school teacher,' '[intern, junior, senior] software engineer,' 'structural engineer,' or 'carpenter.' You can change the state to adjust the pay to a specific location.

In [ ]:
import ipywidgets as widgets
import json, locale

locale.setlocale(locale.LC_ALL, '')

states = dict(json.load(open(settings.STATE_ABBR)))
state_options = dict([(name, states[name]) for name in states])

def estimate_job_salary(Job, State):
    similar_words = j2v.wv.similar_by_vector(job2vec.vectorize(Job), topn=5)
    similar = ', '.join(w[0] for w in similar_words if w[0] not in Job and w[1] > .6) or 'No close similarities, results will likely be inaccurate.'

    result = xgb_pipe.predict(pd.DataFrame({'state':[State], 'job_title':[Job]}))
    hourly_pay = locale.currency(dm.salary_to_hourly(result[0], 'HOURLY'), grouping=True)
    salary_pay = locale.currency(result[0], grouping=True)
    
    return display(HTML(fr'''
<div style="background-color:#ebd5b3;color:black;font-size:17px;font-weight:bold;">
    <p>
        <table>
            <tr>
                <td>Salary pay:</td> 
                <td>{salary_pay}</td>
            </tr>
            <tr>
                <td>Hourly pay:</td> 
                <td>{hourly_pay}</td>
            </tr>
        </table>
    </p>
    <p>
        Similar words: {similar}
    </p>
</div>
                        '''))

widgets.interactive(estimate_job_salary, Job='', State=state_options, placeholder='Write the job title here.')

If you ran all the code but missed the instructions, they are right before the code block above.

#### Resources

Boothe, A. (2023, June 29). Regex for 50 US States - data, code and conversation. Sigpwned.com. https://sigpwned.com/2023/06/29/regex-for-50-us-states/

KON, A. (2024, May 3). LinkedIn Job Postings - 2023. Www.kaggle.com. https://www.kaggle.com/datasets/arshkon/linkedin-job-postings

Paine, J. (n.d.). A python list of all US state abbreviations. Gist. Retrieved July 14, 2024, from https://gist.github.com/JeffPaine/3083347

Plotly, Inc. (2021, September 28). plotly.py. GitHub. https://github.com/plotly/plotly.py

Řehůřek, R., & Sojka, P. (2010, May 1). Software Framework for Topic Modelling with Large Corpora. GitHub. https://github.com/piskvorky/gensim

scikit-learn. (2019, April 15). Scikit-learn/scikit-learn. GitHub. https://github.com/scikit-learn/scikit-learn

U.S. Bureau of Labor Statistics. (2017, October 24). A-Z Index: Occupational Outlook Handbook. Bls.gov. https://www.bls.gov/ooh/a-z-index.htm

U.S. Bureau of Labor Statistics. (2018). Average hours employed people spent working on days worked by day of week. Bls.gov. https://www.bls.gov/charts/american-time-use/emp-by-ftpt-job-edu-h.htm

U.S. Bureau of Labor Statistics. (2019, March 29). Occupational Employment and Wage Statistics. Bls.gov. https://www.bls.gov/oes/current/oes_stru.htm

U.S. Bureau of Labor Statistics. (2021, September 23). Who receives paid vacations? Www.bls.gov. https://www.bls.gov/ebs/factsheets/paid-vacations.htm#:~:text=The%20number%20of%20vacation%20days

United States Court of Appeals for the Second Circuit. (2024). Federal Holidays 2024. Www.ca2.Uscourts.gov. https://www.ca2.uscourts.gov/clerk/calendars/federal_holidays.html

XGBoost Contributors. (2021). GitHub - dmlc/xgboost at stable. GitHub. https://github.com/dmlc/xgboost/tree/stable